In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from dotenv import load_dotenv
import pandas as pd
from datasets import load_dataset
from qdrant_client import models, QdrantClient


from src import load_data, embeddings


c:\Users\manuelalberto.romero\Documents\repos\dslabs\dslab-rag-retrieval\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# https://qdrant.tech/documentation/beginner-tutorials/hybrid-search-fastembed/

# Params

In [3]:
INDEX = "ragquas-sparse-01"
RETRIEVE_K=5

# Environment variables

In [4]:
load_dotenv()

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']


# Clients

In [5]:
client_qdrant = QdrantClient(":memory:") 


# Load Data

In [6]:
df_ragquas = load_data.ragquas()
df_ragquas.head(2)

,topic,answer,question,variant,context_1,context_2,context_3,context_4,context_5,link_1,link_2,link_3,link_4,link_5,text_1,text_2,text_3,text_4,text_5
0,reclamaciones,La opción más fácil y eficaz para reclamar una...,¿Cuál es la forma más fácil de reclamar cuando...,question_1,#1. Airhelp\n\nLa empresa Airhelp fue fundada ...,En AirHelp hemos ayudado a más de 16 millones ...,"MYFLYRIGHT, expertos en derechos de los viajer...",,,https://www.businessinsider.es/mejores-paginas...,https://www.airhelp.com/es/retrasos-de-vuelos/,https://myflyright.com/es/servicios/vuelo-retr...,,,5 páginas donde poder reclamar tus vuelos por ...,Conoce las compensaciones cuando se retrasa tu...,Indemnización retraso vuelo\n\nNavegación por ...,,
1,reclamaciones,La opción más fácil y eficaz para reclamar una...,hola mi vuelo a Alemania se ha retrasado mucho...,question_2,#1. Airhelp\n\nLa empresa Airhelp fue fundada ...,En AirHelp hemos ayudado a más de 16 millones ...,"MYFLYRIGHT, expertos en derechos de los viajer...",,,https://www.businessinsider.es/mejores-paginas...,https://www.airhelp.com/es/retrasos-de-vuelos/,https://myflyright.com/es/servicios/vuelo-retr...,,,5 páginas donde poder reclamar tus vuelos por ...,Conoce las compensaciones cuando se retrasa tu...,Indemnización retraso vuelo\n\nNavegación por ...,,


In [7]:
df_docs = load_data.prepare_ragquas(df_ragquas)
df_docs.info()

Raw shape: (250, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   index     183 non-null    int64 
 1   topic     183 non-null    object
 2   variant   183 non-null    object
 3   question  183 non-null    object
 4   answer    183 non-null    object
 5   j         183 non-null    int64 
 6   context_  183 non-null    object
 7   text_     183 non-null    object
 8   link_     183 non-null    object
 9   uuid      183 non-null    object
dtypes: int64(2), object(8)
memory usage: 14.4+ KB


In [30]:
from src import bm25

corpus = df_docs['text_'].tolist()

encoder = bm25.BM25Vectorizer(k1=1.2, b=0.75).fit(corpus)
corpus_encoded = encoder.transform(corpus) 

c:\Users\manuelalberto.romero\Documents\repos\dslabs\dslab-rag-retrieval\.venv\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [42]:
lst_embeddings = []
for idx in range(corpus_encoded.shape[0]):
    sp_vec = corpus_encoded[idx]
    lst_embeddings.append({
        "text": models.SparseVector(
            indices=sp_vec.indices,
            values=sp_vec.data
            )
        }
    )

print(f"{len(lst_embeddings)=}")

len(lst_embeddings)=183


In [43]:
lst_pts = embeddings.create_index_points(
    lst_embeddings,
    df_docs,
)
print(f"{len(lst_pts)=}")

len(lst_pts)=183


In [44]:
lst_qdrant_pts = embeddings.convert_to_qdrant_points(lst_pts)
print(f"{len(lst_qdrant_pts)=}")

len(lst_qdrant_pts)=183


In [45]:
client_qdrant.create_collection(
    collection_name=INDEX,
    vectors_config={},
    sparse_vectors_config={
        "text": models.SparseVectorParams(),
        }
)

client_qdrant.upsert(
    collection_name=INDEX,
    points=lst_qdrant_pts
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [47]:
_q_text = df_docs.iloc[0:1]['question']

_q_encoded = encoder.transform(_q_text)[0]

_q_vec = models.SparseVector(
    indices=_q_encoded.indices,
    values=_q_encoded.data
    )

print(f"Question: {_q_text}")
embeddings.check_query_qdrant(
        client_qdrant,
        INDEX,
        _q_vec,
        retrieve_k=3,
        using='text'  # passed by *args, **kwargs to query_points()
)



Question: 0    ¿Cuál es la forma más fácil de reclamar cuando...
Name: question, dtype: object
point.id='ada9d109-3f17-4980-81b7-4d537fd44e5f'
point.score=86.91446685791016
Doc: Indemnización retraso vuelo

Navegación por el Retraso en el vuelo

    La indemnización por retraso de vuelo y los derechos en breve
    ¿Tu vuelo se ha retrasado? Condiciones de indemnización
    Indemnización por retraso de vuelo: Derechos de los pasajeros de la UE
    Indemnización por retraso de vuelo: Paquete y viaje de negocios
    Retraso del vuelo: Cómo reclamar una indemnización
    Estado de los vuelos: Estadísticas de la UE sobre retrasos

Indemnizaciones y derechos en caso de retras...
idx: 0, j: 3
------------------------------
point.id='9444be4a-e047-4c1b-97d9-4b5ef6681381'
point.score=83.19611358642578
Doc: Conoce las compensaciones cuando se retrasa tu vuelo

Llegas al aeropuerto y tu vuelo va con retraso, ¿conoces cuáles son tus derechos y si puedes pedir reclamar una compensación? Desde AirHe